In [22]:
from IPython.display import clear_output
import shlex
import matplotlib.pyplot as plt
from brian2 import *
import ptitprince as pt
import seaborn as sns
import matplotlib.colors as mplcol
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from functions import *
import TVB.tvb_model_reference.src.nuu_tools_simulation_human as tools
from Tf_calc.theoretical_tools import *
import subprocess

from TVB.tvb_model_reference.simulation_file.parameter.parameter_M_Berlin_new import Parameter
parameters = Parameter()

import numpy as np
import ast  
from Tf_calc.cell_library import get_neuron_params_double_cell

from math import erf
from functions import *
from Tf_calc.theoretical_tools import convert_params
import json

# Parameter Setup

In [28]:
start_scope()

params = {
    'cell': 'FS-RS',
    'iext': 0.35,
    'time': 4000,
    'Ntot': 100000,
    'kwargs': {
        'use': True,
        'b_e': 85,
        'V_m': -65,
    }
}

CELLS = params['cell']
neuron_params = get_neuron_params_double_cell(CELLS)

kwargs = params['kwargs']

for key in kwargs.keys():
    if key in neuron_params.keys():
        neuron_params[key] = kwargs[key]
    elif key == 'use':
        continue
    else:
        raise Exception(f"Key '{key}' not in the valid keys")
    
locals().update(neuron_params)

cell parameters --NOT-- in SI units


In [ ]:
N1 = int(gei * Ntot)
N2 = int((1-gei) * Ntot)

duration = params['time'] * ms
C = Cm * pF
gL = Gl * nS
tauw = tau_w * ms

I = 0 * nA
Ee = E_e * mV
Ei = E_i * mV

eqs = """
dvm/dt=(gL*(EL-vm)+gL*DeltaT*exp((vm-VT)/DeltaT)-GsynE*(vm-Ee)-GsynI*(vm-Ei)+I-w)/C : volt (unless refractory)
dw/dt=(a*(vm-EL)-w)/tauw : amp
dGsynI/dt = -GsynI/TsynI : siemens
dGsynE/dt = -GsynE/TsynE : siemens
TsynI:second
TsynE:second
Vr:volt
b:amp
a:siemens
DeltaT:volt
Vcut:volt
VT:volt
EL:volt
"""

G_inh = NeuronGroup(N1, model=eqs, threshold='vm > Vcut',refractory=5*ms,
                     reset="vm = Vr; w += b", method='heun')
G_inh.vm = V_m * mV 
G_inh.a = a_i * nS 
G_inh.w = G_inh.a * (G_inh.vm - G_inh.EL)
G_inh.Vr = V_r * mV  
G_inh.TsynI = tau_i * ms  
G_inh.TsynE = tau_e * ms  
G_inh.b = b_i * pA
G_inh.DeltaT = delta_i * mV
G_inh.VT = V_th * mV
G_inh.Vcut = V_cut * mV
G_inh.EL = EL_i * mV

# Population 2 - Regular Spiking

G_exc = NeuronGroup(N2, model=eqs, threshold='vm > Vcut',refractory=5*ms,
                     reset="vm = Vr; w += b", method='heun')
G_exc.vm = V_m*mV
G_exc.a = a_e * nS 
G_exc.w = G_exc.a * (G_exc.vm - G_exc.EL)
G_exc.Vr = V_r*mV
G_exc.TsynI =tau_i*ms
G_exc.TsynE =tau_e*ms
G_exc.b=b_e*pA
G_exc.DeltaT=delta_e*mV
G_exc.VT=V_th*mV
G_exc.Vcut = V_cut * mV
G_exc.EL=EL_e*mV

In [30]:
Qi=Q_i*nS
Qe=Q_e*nS
P_ed=PoissonGroup(N2, rates=params['iext']*Hz)

prbC=p_con #0.05

S_12 = Synapses(G_inh, G_exc, on_pre='GsynI_post+=Qi') #'v_post -= 1.*mV')
S_12.connect('i!=j', p=prbC)

S_11 = Synapses(G_inh, G_inh, on_pre='GsynI_post+=Qi')
S_11.connect('i!=j',p=prbC)

S_21 = Synapses(G_exc, G_inh, on_pre='GsynE_post+=Qe')
S_21.connect('i!=j',p=prbC)

S_22 = Synapses(G_exc, G_exc, on_pre='GsynE_post+=Qe')
S_22.connect('i!=j', p=prbC)

S_ed_in = Synapses(P_ed, G_inh, on_pre='GsynE_post+=Qe')
S_ed_in.connect(p=prbC)

S_ed_ex = Synapses(P_ed, G_exc, on_pre='GsynE_post+=Qe')
S_ed_ex.connect(p=prbC)

PgroupE = NeuronGroup(1, 'P:amp', method='heun')

PE=Synapses(G_exc, PgroupE, 'P_post = w_pre : amp (summed)')
PE.connect(p=1)
P2mon = StateMonitor(PgroupE, 'P', record=0)

# Recording tools -------------------------------------------------------------------------------
rec1=2000
rec2=8000

M1G_inh = SpikeMonitor(G_inh)
FRG_inh = PopulationRateMonitor(G_inh)
M1G_exc = SpikeMonitor(G_exc)
FRG_exc = PopulationRateMonitor(G_exc)

run(duration)


WARNING    'EL' is an internal variable of group 'neurongroup_6', but also exists in the run namespace with the value -64. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'a' is an internal variable of group 'neurongroup_6', but also exists in the run namespace with the value 0. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'EL' is an internal variable of group 'neurongroup_8', but also exists in the run namespace with the value -64. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'a' is an internal variable of group 'neurongroup_8', but also exists in the run namespace with the value 0. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'b' is an internal variable of group 'neurongroup_6', but also exists in the run namespace with the value 85. The internal variable wil

In [ ]:
RasG_inh = np.array([M1G_inh.t / ms, [i + N2 for i in M1G_inh.i]])
RasG_exc = np.array([M1G_exc.t / ms, M1G_exc.i])
if AmpStim > 0:
    time_array = np.arange(int(TotTime / DT)) * DT
    input_bin = bin_array(np.array(test_input), 5, time_array)
else:
    input_bin = np.full(1, np.nan)
TimBinned, popRateG_exc, popRateG_inh, Pu = prepare_FR(TotTime, DT, FRG_exc, FRG_inh, P2mon, BIN=5)
if save_path:
    try:
        os.listdir(save_path)
    except:
        os.makedirs(save_path)
    
    if save_mean:
        print("Saving the mean")
        # print("Exc=", np.mean(popRateG_exc[int(len(popRateG_exc)/2):]), "Inh=",np.mean(popRateG_inh[int(len(popRateG_inh)/2):]))
        np.save(save_path + f'{CELLS}_mean_exc_amp_{AmpStim}.npy', np.array([np.mean(popRateG_exc[int(len(popRateG_exc)/2):]),AmpStim, neuron_params], dtype=object))
        np.save(save_path + f'{CELLS}_mean_inh_amp_{AmpStim}.npy', np.array([np.mean(popRateG_inh[int(len(popRateG_inh)/2):]), AmpStim, neuron_params], dtype=object))
    if save_all:
        print("Saving the whole simulation")
        np.save(save_path + f'{CELLS}_inh_amp_{AmpStim}.npy', np.array([popRateG_inh, AmpStim, neuron_params], dtype=object))
        np.save(save_path + f'{CELLS}_exc_amp_{AmpStim}.npy', np.array([popRateG_exc,AmpStim, neuron_params], dtype=object))

# # ----- Raster plot + mean adaptation ------

fig, axes = plt.subplots(2, 1, figsize=(5, 8))

plot_raster_meanFR(RasG_inh, RasG_exc, TimBinned, popRateG_inh, popRateG_exc, Pu, axes, sim_name, input_bin)


print(f" done")
plt.show()

ValueError: cannot reshape array of size 40000 into shape (19999,50)